# Resources


*  https://github.com/LIAAD/yake
*  https://pypi.org/project/PyArabic/



# Get Data (Text Files)


In [ ]:
import numpy as np
import pandas as pd
import pdb
import string
import os
import re

from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer

In [ ]:
!wget https://liquidtelecom.dl.sourceforge.net/project/easc-corpus/EASC/EASC.zip

--2019-09-03 08:17:03--  https://liquidtelecom.dl.sourceforge.net/project/easc-corpus/EASC/EASC.zip
Resolving liquidtelecom.dl.sourceforge.net (liquidtelecom.dl.sourceforge.net)... 197.155.77.8
Connecting to liquidtelecom.dl.sourceforge.net (liquidtelecom.dl.sourceforge.net)|197.155.77.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1519413 (1.4M) [application/octet-stream]
Saving to: ‘EASC.zip’

EASC.zip            100%[===================>]   1.45M  1.95MB/s    in 0.7s    

2019-09-03 08:17:05 (1.95 MB/s) - ‘EASC.zip’ saved [1519413/1519413]



In [ ]:
!unzip EASC.zip

Archive:  EASC.zip
  inflating: EASC/COPYRIGHT          
   creating: EASC/EASC-ISO-Arabic/
   creating: EASC/EASC-ISO-Arabic/Articles/
   creating: EASC/EASC-ISO-Arabic/Articles/Topic1/
  inflating: EASC/EASC-ISO-Arabic/Articles/Topic1/Art and Music (1).txt  
   creating: EASC/EASC-ISO-Arabic/Articles/Topic10/
  inflating: EASC/EASC-ISO-Arabic/Articles/Topic10/Art and Music (10).txt  
   creating: EASC/EASC-ISO-Arabic/Articles/Topic100/
  inflating: EASC/EASC-ISO-Arabic/Articles/Topic100/politics (16).TXT  
   creating: EASC/EASC-ISO-Arabic/Articles/Topic101/
  inflating: EASC/EASC-ISO-Arabic/Articles/Topic101/politics (17).TXT  
   creating: EASC/EASC-ISO-Arabic/Articles/Topic102/
  inflating: EASC/EASC-ISO-Arabic/Articles/Topic102/politics (18).TXT  
   creating: EASC/EASC-ISO-Arabic/Articles/Topic103/
  inflating: EASC/EASC-ISO-Arabic/Articles/Topic103/politics (19).txt  
   creating: EASC/EASC-ISO-Arabic/Articles/Topic104/
  inflating: EASC/EASC-ISO-Arabic/Articles/Topic104/politi

In [ ]:
!ls

EASC  EASC.zip	sample_data


In [ ]:
cd EASC

/content/EASC


In [ ]:
!ls 

COPYRIGHT  EASC-ISO-Arabic  EASC-UTF-8	README	SOURCES.CSV


In [ ]:
!wget https://github.com/mohataher/arabic-stop-words/blob/master/list.txt

--2019-09-03 08:17:16--  https://github.com/mohataher/arabic-stop-words/blob/master/list.txt
Resolving github.com (github.com)... 192.30.253.113
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘list.txt’

list.txt                [  <=>               ] 204.10K   638KB/s    in 0.3s    

2019-09-03 08:17:17 (638 KB/s) - ‘list.txt’ saved [208999]



In [ ]:
!cat EASC/EASC-UTF-8/MTurk/Topic1/D0101.M.250.A.1.A 

cat: EASC/EASC-UTF-8/MTurk/Topic1/D0101.M.250.A.1.A: No such file or directory


In [ ]:
!cat EASC/EASC-UTF-8/MTurk/Topic1/D0101.M.250.A.2.B

cat: EASC/EASC-UTF-8/MTurk/Topic1/D0101.M.250.A.2.B: No such file or directory


In [ ]:
cd  EASC/EASC-UTF-8/MTurk/Topic1/

[Errno 2] No such file or directory: 'EASC/EASC-UTF-8/MTurk/Topic1/'
/content/EASC


In [ ]:
!find -maxdepth 1 -type f -ls -exec file -b {} \;

 19136545      4 -rw-r--r--   1 root     root         2718 Feb 20  2010 ./COPYRIGHT
find: ‘file’: No such file or directory
 19139000      4 -rw-r--r--   1 root     root         2317 Feb 20  2010 ./README
find: ‘file’: No such file or directory
 19139001     16 -rw-r--r--   1 root     root        13078 Feb 20  2010 ./SOURCES.CSV
find: ‘file’: No such file or directory
 19139005    208 -rw-r--r--   1 root     root       208999 Sep  3 08:17 ./list.txt
find: ‘file’: No such file or directory


In [ ]:
!pwd


/content/EASC


# Build the Dataset

In [ ]:
train_dir_name = '/content/EASC/EASC-UTF-8/Articles/'
test_dir_name = '/content/EASC/EASC-UTF-8/MTurk/'
def getListOfFiles(dirName):
    listOfFile = os.listdir(dirName)
    allFiles = list()
    for entry in listOfFile:
        fullPath = os.path.join(dirName, entry) 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

In [ ]:
train_files_path = getListOfFiles(train_dir_name)

In [ ]:
train_files_path[:15]

['/content/EASC/EASC-UTF-8/Articles/Topic69/health (2).TXT',
 '/content/EASC/EASC-UTF-8/Articles/Topic67/finance (17).txt',
 '/content/EASC/EASC-UTF-8/Articles/Topic106/religion (1).txt',
 '/content/EASC/EASC-UTF-8/Articles/Topic143/tourisms (4).txt',
 '/content/EASC/EASC-UTF-8/Articles/Topic95/politics (11).txt',
 '/content/EASC/EASC-UTF-8/Articles/Topic3/Art and Music (3).txt',
 '/content/EASC/EASC-UTF-8/Articles/Topic21/environment (5).txt',
 '/content/EASC/EASC-UTF-8/Articles/Topic17/education (7).txt',
 '/content/EASC/EASC-UTF-8/Articles/Topic14/education (4).txt',
 '/content/EASC/EASC-UTF-8/Articles/Topic4/Art and Music (4).txt',
 '/content/EASC/EASC-UTF-8/Articles/Topic65/finance (15).TXT',
 '/content/EASC/EASC-UTF-8/Articles/Topic93/politics (9).txt',
 '/content/EASC/EASC-UTF-8/Articles/Topic96/politics (12).txt',
 '/content/EASC/EASC-UTF-8/Articles/Topic109/religion (4).txt',
 '/content/EASC/EASC-UTF-8/Articles/Topic24/environment (8).TXT']

In [ ]:
len(train_files_path)

153

In [ ]:
test_files_path = getListOfFiles(test_dir_name)

In [ ]:
test_files_path[:15]

['/content/EASC/EASC-UTF-8/MTurk/Topic69/D0502.M.250.A.5.E',
 '/content/EASC/EASC-UTF-8/MTurk/Topic69/D0502.M.250.A.4.D',
 '/content/EASC/EASC-UTF-8/MTurk/Topic69/D0502.M.250.A.2.B',
 '/content/EASC/EASC-UTF-8/MTurk/Topic69/D0502.M.250.A.1.A',
 '/content/EASC/EASC-UTF-8/MTurk/Topic69/D0502.M.250.A.3.C',
 '/content/EASC/EASC-UTF-8/MTurk/Topic67/D04017.M.250.A.3.C',
 '/content/EASC/EASC-UTF-8/MTurk/Topic67/D04017.M.250.A.2.B',
 '/content/EASC/EASC-UTF-8/MTurk/Topic67/D04017.M.250.A.1.A',
 '/content/EASC/EASC-UTF-8/MTurk/Topic67/D04017.M.250.A.4.D',
 '/content/EASC/EASC-UTF-8/MTurk/Topic67/D04017.M.250.A.5.E',
 '/content/EASC/EASC-UTF-8/MTurk/Topic106/D0701.M.250.A.3.C',
 '/content/EASC/EASC-UTF-8/MTurk/Topic106/D0701.M.250.A.5.E',
 '/content/EASC/EASC-UTF-8/MTurk/Topic106/D0701.M.250.A.4.D',
 '/content/EASC/EASC-UTF-8/MTurk/Topic106/D0701.M.250.A.1.A',
 '/content/EASC/EASC-UTF-8/MTurk/Topic106/D0701.M.250.A.2.B']

In [ ]:
len(test_files_path)


765

In [ ]:
len(train_files_path) * 5

765

In [ ]:
data = pd.DataFrame(columns = ['Orignal' ,'Summary1' ,'Summary2' ,'Summary3' ,'Summary4' ,'Summary5'])

In [ ]:
j = 0
for i in range(len(train_files_path)) :
  train_path = train_files_path[i]
  test_path = test_files_path[j : j + 5]
  j+=5
  
  train_file = open(train_path , 'r' ,encoding = 'utf-8').read()
  sum1 =  open(test_path[0] , 'r' ,encoding = 'utf-8').read()
  sum2 =  open(test_path[1] , 'r' ,encoding = 'utf-8').read()
  sum3 =  open(test_path[2] , 'r' ,encoding = 'utf-8').read()
  sum4 =  open(test_path[3] , 'r' ,encoding = 'utf-8').read()
  sum5 =  open(test_path[4] , 'r' ,encoding = 'utf-8').read()
  
  row_list = []
  row_list.extend((train_file, sum1, sum2,sum3,sum4,sum5))
  data.loc[len(data)] = row_list

  
  
  
  

In [ ]:
data.head()

,Orignal,Summary1,Summary2,Summary3,Summary4,Summary5
0,﻿الإثنا عشري(بالإنجليزية: Duodenum) أو العفج ه...,يقوم الاثنا عشر بوظيفة أساسية في عملية الهضم.\...,الإثنا عشري بالإنجليزية: Duodenum أو العفج هو ...,يقوم الاثنا عشر بوظيفة أساسية في عملية الهضم.,المواد النشوية التي لم يتم تفكيكها في الفم و ا...,الإثنا عشري بالإنجليزية: Duodenum أو العفج هو ...
1,﻿في وحدة خدمات المشتركين بكهرباء عسير، صباح ال...,في وحدة خدمات المشتركين بكهرباء عسير، صباح الي...,في وحدة خدمات المشتركين بكهرباء عسير، صباح الي...,في وحدة خدمات المشتركين بكهرباء عسير، صباح الي...,في وحدة خدمات المشتركين بكهرباء عسير، صباح الي...,في وحدة خدمات المشتركين بكهرباء عسير، صباح الي...
2,﻿المسيحية هي إحدى الديانات السماوية التي يعتبر...,المسيحية هي احدى الديانات السماوية التي يعتبر ...,المسيحية هي احدى الديانات السماوية التي يعتبر ...,المسيحية هي احدى الديانات السماوية التي يعتبر ...,تعتبر المسيحية اكثر الديانات اتباعا في العالم...,المسيحية هي احدى الديانات السماوية التي يعتبر ...
3,﻿تعتبر القدس مدينة مقدسة بالنسبة للديانات التو...,تعتبر القدس مدينة مقدسة بالنسبة للديانات التوح...,اول اسم ثابت لمدينة القدس هو اورسالم قبل خمسة ...,تعتبر القدس مدينة مقدسة بالنسبة للديانات التوح...,وتذكر في تاريخ اليهود والمسيحيين باعتبارها ال...,تعتبر القدس مدينة مقدسة بالنسبة للديانات التوح...
4,﻿أعلنت مصادر رسمية أن الحكومة الانتقالية برئاس...,اعلنت مصادر رسمية ان الحكومة الانتقالية برئاسة...,اعلنت مصادر رسمية ان الحكومة الانتقالية برئاسة...,اعلنت مصادر رسمية ان الحكومة الانتقالية برئاسة...,على صعيد آخر اعلن قيادي في حركة الوفاق الوطني...,اعلنت مصادر رسمية ان الحكومة الانتقالية برئاسة...


In [ ]:
data.shape

(153, 6)

# Preprocessing


In [ ]:
list_dir = '/content/EASC/list.txt'

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import unicodedata as ud
for c in 'abc:. . قيم ؟': print((c, ud.category(c)))

('a', 'Ll')
('b', 'Ll')
('c', 'Ll')
(':', 'Po')
('.', 'Po')
(' ', 'Zs')
('.', 'Po')
(' ', 'Zs')
('ق', 'Lo')
('ي', 'Lo')
('م', 'Lo')
(' ', 'Zs')
('؟', 'Po')


In [ ]:
t = 'abc:. . قيم ؟'

In [ ]:
lst = []
for c in t :
  if c == '.' : lst.append(c)
  if not ud.category(c).startswith('P') : lst.append(c)
t2 = ''.join(lst)
t2

'abc. . قيم '

In [ ]:
ud.category(c).startswith('P')

True

In [ ]:
def noramlize(text):
    text = re.sub(r"[إأٱآا]", "ا", text)
    text = re.sub(r"ى", "ي", text)
    text = re.sub(r"ؤ", "ء", text)
    text = re.sub(r"ئ", "ء", text)
    #text = re.sub(r"ه", "ة", text) 
  
    
    #text = re.sub(r'[^ا-ي ]', "", text)
    
    

    noise = re.compile(""" ّ      | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, '', text)
    
    lst = []
    for c in text :
      if c == '.' : lst.append(c)
      if not ud.category(c).startswith('P') : lst.append(c)
    text = ''.join(lst)
        
    #text = ''.join(c for c in text if not ud.category(c).startswith('P') and c != '.')
    #remove = '!"#$%&\'()*+،-./:;<=>?@[\\]^_`{|}~'
    #text = text.strip(remove)
    #text = text.strip(string.punctuation)
    
    
    return text

In [ ]:
def stopWordRmove(text):
    ar_stop_list = open(list_dir, "r")
    stop_words = ar_stop_list.read().split('\n')
    
    text = text.replace('\n' ," ")
    text = text.replace("\"" ," ").strip("''")
    text = text.replace('\ufeff' ," ")
    text = text.replace("``" ," ").strip()
    #text = noramlize(text)
    
    
    words = word_tokenize(text)
    needed_words = []
    for w in words:
        if w not in (stop_words) and w != '\ufeff':
            needed_words.append(w)
            
    filtered_sentence = " ".join(needed_words)
      
    return filtered_sentence

In [ ]:
def stopWordRmoveSentence(text):
    ar_stop_list = open(list_dir, "r")
    stop_words = ar_stop_list.read().split('\n')
    sentence_list = []
    org_regex = "[.]+"
    
    text = text.replace('\n' ," ")
    text = text.replace("\"" ," ").strip("''")
    text = text.replace("``" ," ").strip()
    #text = noramlize(text)
    
    sentences = [sentence for sentence in  re.split(org_regex,text)]
    for sentence in sentences :
      #pdb.set_trace()
      words = word_tokenize(sentence)
      needed_words = []
      for w in words:
          if w not in (stop_words) and w != '\ufeff':
              needed_words.append(w)
      filtered_sentence = ""
      if len(needed_words) > 1 : 
        filtered_sentence = " ".join(needed_words)
        sentence_list.append(needed_words)
      
    return sentence_list

In [ ]:
def stemming(text):
    st = ISRIStemmer()
    stemmed_words = []
    words = word_tokenize(text)
    for w in words:
        stemmed_words.append(st.stem(w))
    stemmed_sentence = " ".join(stemmed_words)
    return stemmed_sentence
  



In [ ]:
def stemmingSentence(text):
    st = ISRIStemmer()
    org_regex = "[.]+"
    #re.sub(r'([?!,.]+)',r' \1 ',s) split on ther characters
    #regex = "/[^.\s]+(?:\s+\([^)]*\))?/g"
    sentence_list = []
    sentences = [sentence for sentence in  re.split(org_regex,text)]

    
    for sentence in sentences :
      words = word_tokenize(sentence)
      stemmed_words = []
      for w in words:
          stemmed_words.append(st.stem(w))
          
      stemmed_sentence = ""
      if len(stemmed_words) > 1 :
        stemmed_sentence = " ".join(stemmed_words)
        sentence_list.append(stemmed_sentence)
       
    return sentence_list  

In [ ]:
text = data['Orignal'][83]
text = noramlize(text)
text = stopWordRmove(text)
stemmingSentence(text)

['حدد عنصر كيمياء فلز من قدم عدن كشف رمز له رمز Fe عدد ذري 26 وفي جدل دور يقع حدد في جمع ثمن دور ربع وهو عنصر ضرر لحا انس كون دخل في ركب هيموجلوب لدم كذل لحا نبت كون دخل في ركب كلوروفيل',
 'و دخل في كل شيء حتل حدد ركز ربع من حيث وجد عنصر في قشر رضي وهو فلز قبل طرق سحب دخل في صنع عدد من سبء',
 'حوي نيز سقط علي ارض علي كمي من حدد قد تصل الي 90 من كتل نيز',
 'شبك بلر حدد علي هيء كعب وزع علي كل زاي من زيا ذرة حدد ثمن ذرت تقع اسع في ركز كعب اوتونيوم بنء فرد في دين روكسل مثل وذج شبك بلر حدد كبر 165 لير مرة جاء بنء رمز عظم حدد ودر في حية بشر',
 'يعد حدد اقي فلز علي طلق كثر همي غرض هندس شرط حمي من صدا اي فعل مع كسج وهن عدة طرق لحم حدد من صداوابسط منع تمس كسج رطب عن حدد وذل غلف حدد بمد عزل مثل خدم صبغ او عزل',
 'من فضل طرق خدم في صنع هي خدم نظم حمي كثد لحم حدد من صدا تكل',
 'رمز حدد في ثقف عمة الي قوة و صلب ومن ذلك قول لا يفل حدد الا حدد و قول ضرب بيد من حدد',
 'شكل حديداحدي قعد التي تقم عليها حضر',
 'و رجع كون كثثر فاز خدا الي خصه قيمةو الي وفر خما و سهل وصل اليها',
 'ان حدد وجد في عظم قشر رضي

In [ ]:
text = data['Orignal'][83]
text = noramlize(text)
text = stopWordRmove(text)
text

'الحديد عنصر كيمياءي وفلز من اقدم المعادن المكتشفة يرمز له بالرمز Fe وعدده الذري 26 وفي الجدول الدوري يقع الحديد في المجموعة الثامنة والدورة الرابعة وهو عنصر ضروري لحياة الانسان كونه يدخل في تركيب هيموجلوبين الدم وكذلك لحياة النباتات كونه يدخل في تركيب الكلوروفيل .و يدخل في كل شيء ويحتل الحديد المركز الرابع من حيث وجود العناصر في القشرة الارضية وهو فلز قابل للطرق والسحب ويدخل في صناعة العديد من السباءك .تحتوي النيازك الساقطة علي الارض علي كميات من الحديد قد تصل الي 90 من كتلة النيازك.الشبكة البلورية للحديد علي هيءة مكعب تتوزع علي كل زاوية من زواياه ذرة حديد ثمانية ذرات وتقع تاسعة في مركز المكعب والاوتونيوم البناء الفريد في مدينة بروكسل يمثل نموذج الشبكة البلورية للحديد مكبرا 165 مليار مرة جاء البناء رمزا لعظمة الحديد ودوره في حياة البشرية.يعد الحديد اقوي الفلزات علي الاطلاق واكثرها اهمية للاغراض الهندسية شرط حمايته من الصدا اي التفاعل مع الاكسجين وهناك عدة طرق لحماية الحديد من الصداوابسطها منع تماس الاكسجين والرطوبة عن الحديد وذلك بتغليف الحديد بمادة عازلة مثل استخدام الاصباغ او عوازل 

In [ ]:
data[data['Orignal'].str.contains("العربية المتحدة")]

,Orignal,Summary1,Summary2,Summary3,Summary4,Summary5
28,﻿تم حفر أول بئر للبترول في الصين في القرن الرا...,تم حفر اول بئر للبترول في الصين في القرن الراب...,وكان يتم احراق الزيت لتبخير الماء المالح لانت...,وكان يتم احراق الزيت لتبخير الماء المالح لانت...,تم حفر اول بئر للبترول في الصين في القرن الراب...,وكان يتم احراق الزيت لتبخير الماء المالح لانت...
57,﻿هوكي الجليد هي رياضة يتبارى فيها فريقان على م...,هوكي الجليد هي رياضة يتبارى فيها فريقان على مس...,يضع اللاعبون زلاجات على اقدامهم لكي يتمكنوا م...,هوكي الجليد هي رياضة يتبارى فيها فريقان على مس...,هوكي الجليد هي رياضة يتبارى فيها فريقان على مس...,لكل فريق ستة لاعبين اثناء اللعب، ولكن عدد الف...
72,﻿إمارة دبي هي ثاني الإمارات المكونة لدولة الإم...,دبي هي العاصمة الاقتصادية للامارات العربية الم...,امارة دبي هي ثاني الامارات المكونة لدولة الاما...,امارة دبي هي ثاني الامارات المكونة لدولة الاما...,وتقع امارة دبي بين امارتي ابو ظبي والشارقة.,امارة دبي هي ثاني الامارات المكونة لدولة الاما...
118,﻿حرب اليمن أو حرب شمال اليمن الأهلية هي حرب دا...,﻿حرب اليمن أو حرب شمال اليمن الأهلية هي حرب دا...,فكانت الدول العربية مستقطبة بين المعسكرين الشر...,حرب اليمن أو حرب شمال اليمن الأهلية هي حرب دار...,وقد سيطرت الفصائل الجمهورية على الحكم في نهاية...,حرب اليمن أو حرب شمال اليمن الأهلية هي حرب دار...


In [ ]:
stemmingSentence(data.iloc[113,0])

['\ufeffبعد نشر دين سلم بدأ حكم سلم لسط لمد 1300 سنة',
 'لسط قدس سلم لأن الل امر سيد حمد عليه صلة و سلم بأن تكون قدس قبل اول سلم ( تجه الذي يجه سلم عند يصل ) ولأ صلي الل عليه سلم صعد في ليل سرء و عرج في رحل الى سمء من دين نور الى سجد قصى ضفة سجد قبة صخر بعد ذلك',
 'صبح قدس دين قدس ثلث سلم',
 'حكم سلم لم جبر دين على فلسطينيين، خلل كثر من قرن تحل غلب الى سلم',
 'سيح يهد باق عبر اهل كتب',
 'سمح لهم يطر قلة ذات في جمع وضم لهم امن وحر في عبد',
 'مثل هذا سمح كان ندر في ارخ ادي و في ارخ لسط',
 'كثر لسط تبن عرب ثقف سلم',
 'لسط فاد من جار مع امبراطوري جار ومن اهم دين خلل حكم امي سلم في دمشق',
 'عند نقل سلط الى غدد مع عبس في 750، صبح لسط همل',
 'عنت بلد بعد ذلك من ضطراب هيم عاقب من قبل سلجوقيين، فاطميين، غزو صلب',
 'شرك طين، على اية حل، في مجد حضر اسلامية، عند متع علم سلم عصر ذهب من علم، فن، فة، أدب',
 'حيث بدأ سلم علم علم يون بدأ طرق جدد في عدة حقول، بعد سنن كان لكل ذلك سهم كبر في عصر نهض في ورب',
 'عرب كنع كان اول سكن عرف لسط',
 'خلل الف ثلث قبل يلد وقد صبح دني يعش في دول ومد منها ارح',
 'طور 

In [ ]:
dubai = data.iloc[113,0]
dubai = noramlize(dubai)
dubai = stopWordRmove(dubai)
stemmingSentence(dubai)

['بعد نشر دين سلم بدا حكم سلم لسط لمد 1300 سنة',
 'لسط قدس سلم لان الل امر سيد حمد عليه صلة و سلم بان تكون قدس قبل اول سلم تجه الذي يجه سلم عند يصل ولن صلي الل عليه سلم صعد في ليل سرء و عرج في رحل الي سمء من دين نور الي سجد اقص ضفة سجد قبة صخر بعد ذلك',
 'صبح قدس دين قدس ثلث سلم',
 'حكم سلم لم جبر دين علي لسط خلل كثر من قرن تحل غلب الي سلم',
 'سيح يهد باق عبر اهل كتب',
 'سمح لهم يطر قلة ذات في جمع وضم لهم امن وحر في عبد',
 'مثل هذا سمح كان ندر في ارخ ادي و في ارخ لسط',
 'كثر لسط تبن عرب ثقف سلم',
 'لسط فاد من جار مع امبراطوري جار ومن اهم دين خلل حكم امي سلم في دمشق',
 'عند نقل سلط الي غدد مع عبس في 750 صبح لسط همل',
 'عنت بلد بعد ذلك من ضطراب هيم عاقب من قبل لجق فطم غزو صلب',
 'شرك لسط علي اية حال في مجد حضر سلم عند متع علم سلم عصر ذهب من علم فن لسف ودب',
 'حيث بدا سلم علم علم يون وبد طرق جدد في عدة حقل بعد سنن كان لكل ذلك سهم كبر في عصر نهض في ورب',
 'عرب كنع كان اول سكن عرف لسط',
 'خلل الف ثلث قبل يلد وقد صبح دني يعش في دول ومد منها ارح',
 'طور بجد ومن شقت نظم كتب اخر',
 'وقع لسط في 

In [ ]:
len(stemmingSentence(dubai))

36

In [ ]:
text = data['Orignal'][0]
regex = "/[^.\s]+(?:\s+\([^)]*\))?/g"
org_regex = "[.]+"
text = text.replace('\n' ," ")
sentences = [sentence for sentence in  re.split(org_regex,text)]
sentences

['\ufeffالإثنا عشري(بالإنجليزية: Duodenum) أو العفج هو أحد أجزاء القناة الهضمية وأول جزء من الأمعاء الدقيقة يصل بين المعدة و الصائم',
 ' طوله 25-30 سم وله أربعة قطع تشبه المضلع تنفتح في القطعة الثانية منه كلا من القناتين الصفراوية والمعثكلية ويفصله عن المعدة البواب',
 'يقوم الاثنا عشر بوظيفة أساسية في عملية الهضم',
 'المواد النشوية التي لم يتم تفكيكها في الفم و المعدة يقوم الثنى عشر بإكمال عملية تفكيكها بمساعدة أنزيمات يفرزها البنكرياس كذلك مقاطع البروتينات المسماه ببتونات التي تفككت من البروتينات في المعدة تستمر عملية هضمها في الاثنى عشر بمساعدة أنزيمات أساسية',
 ' كذلك يتم تحليل وهضم الدهون وكذا تتحلل وتهضم في الاثنى عشر بواسطة أنزيمات تصل من البنكرياس الفولماريات مثل المواد المضافة للغذاء كاللون والمنكهات وما شابه التي لم تحلل في المعدة والفم',
 'المواد التي تصل من البنكرياس والكبد و المرارة تساعد على تحليل بعض مركبات الغذاء كيميائيا والمواد التي يتم تحليلها كيميائيا هي:بروتينات كروهيدرات ودهون بالاضافة لذلك يتم امتصاص مياة وموادمعدنية الى الدم علما بان المواد المعدنية لا يحل بها تغ

In [ ]:
stopWordRmoveSentence(text)

[['\ufeffالإثنا',
  'عشري',
  '(',
  'بالإنجليزية',
  ':',
  'Duodenum',
  ')',
  'أو',
  'العفج',
  'هو',
  'أحد',
  'أجزاء',
  'القناة',
  'الهضمية',
  'وأول',
  'جزء',
  'من',
  'الأمعاء',
  'الدقيقة',
  'يصل',
  'بين',
  'المعدة',
  'و',
  'الصائم'],
 ['طوله',
  '25-30',
  'سم',
  'وله',
  'أربعة',
  'قطع',
  'تشبه',
  'المضلع',
  'تنفتح',
  'في',
  'القطعة',
  'الثانية',
  'منه',
  'كلا',
  'من',
  'القناتين',
  'الصفراوية',
  'والمعثكلية',
  'ويفصله',
  'عن',
  'المعدة',
  'البواب'],
 ['يقوم', 'الاثنا', 'عشر', 'بوظيفة', 'أساسية', 'في', 'عملية', 'الهضم'],
 ['المواد',
  'النشوية',
  'التي',
  'لم',
  'يتم',
  'تفكيكها',
  'في',
  'الفم',
  'و',
  'المعدة',
  'يقوم',
  'الثنى',
  'عشر',
  'بإكمال',
  'عملية',
  'تفكيكها',
  'بمساعدة',
  'أنزيمات',
  'يفرزها',
  'البنكرياس',
  'كذلك',
  'مقاطع',
  'البروتينات',
  'المسماه',
  'ببتونات',
  'التي',
  'تفككت',
  'من',
  'البروتينات',
  'في',
  'المعدة',
  'تستمر',
  'عملية',
  'هضمها',
  'في',
  'الاثنى',
  'عشر',
  'بمساعدة',
  'أنزيما

In [ ]:
def preprocess(org_df):
    df = org_df.copy()
    ids_col = df.filter(regex='[Ii]d$').columns.values
    df.drop(ids_col,axis =1 ,inplace = True)
    
    for col in df.columns :
        if df[col].dtype == np.object:
            df[col] = df[col].apply(lambda x : stopWordRmove(x))
            df[col] = df[col].apply(lambda x : noramlize(x))
            df[col] = df[col].apply(lambda x : stemming(x))
                 
    return df
            

In [ ]:
def preprocessSentence(org_df):
    df = org_df.copy()
    ids_col = df.filter(regex='[Ii]d$').columns.values
    df.drop(ids_col,axis =1 ,inplace = True)
    
    for col in df.columns :
        if df[col].dtype == np.object:
            df[col] = df[col].apply(lambda x : stopWordRmove(x))
            df[col] = df[col].apply(lambda x : noramlize(x))
            df[col] = df[col].apply(lambda x : stemmingSentence(x))
                 
    return df

In [ ]:
new_df = preprocess(data)
new_df.head()

,Orignal,Summary1,Summary2,Summary3,Summary4,Summary5
0,اثن عشر انجليزية Duodenum او عفج هو احد جزء قن...,يقم اثن عشر وظف سسي في عمل هضم . كذلك قبل اثن ...,اثن عشر انجليزية Duodenum او عفج هو احد جزء قن...,يقم اثن عشر وظف سسي في عمل هضم .,واد نشي التي لم يتم فكك في لفم و عدة يقم ثني ع...,اثن عشر انجليزية Duodenum او عفج هو احد جزء قن...
1,في وحد خدم شرك كهرباء عسر صبح اليوم تقل رقم ال...,في وحد خدم شرك كهرباء عسر صبح اليوم تقل رقم ال...,في وحد خدم شرك كهرباء عسر صبح اليوم تقل رقم ال...,في وحد خدم شرك كهرباء عسر صبح اليوم تقل رقم ال...,في وحد خدم شرك كهرباء عسر صبح اليوم تقل رقم ال...,في وحد خدم شرك كهرباء عسر صبح اليوم تقل رقم ال...
2,سيح هي احد دين سمو التي عبر يسع سيح شخص سسي في...,سيح هي احد دين سمو التي عبر يسع سيح شخص سسي في...,سيح هي احد دين سمو التي عبر يسع سيح شخص سسي في...,سيح هي احد دين سمو التي عبر يسع سيح شخص سسي في...,عبر سيح كثر دين تبع في علم عدد تبع بلغ 2 . 1 ب...,سيح هي احد دين سمو التي عبر يسع سيح شخص سسي في...
3,عبر قدس دين قدس نسب دين وحد ثلث سلم سيح يهد نس...,عبر قدس دين قدس نسب دين وحد ثلث سلم سيح يهد نس...,اول اسم ثبت لمد قدس هو رسلم قبل خمس الف عام وي...,عبر قدس دين قدس نسب دين وحد ثلث سلم سيح يهد نس...,ذكر في ارخ يهد سيح عبر ركز لكي دين ممل يهذ ارخ...,عبر قدس دين قدس نسب دين وحد ثلث سلم سيح يهد نس...
4,اعل صدر رسم ان حكم انتقالية رءس دكتور راهيم جع...,اعل صدر رسم ان حكم انتقالية رءس دكتور راهيم جع...,اعل صدر رسم ان حكم انتقالية رءس دكتور راهيم جع...,اعل صدر رسم ان حكم انتقالية رءس دكتور راهيم جع...,علي صعد اخر اعل قيد في حرك وفق وطن عرق ل وطن و...,اعل صدر رسم ان حكم انتقالية رءس دكتور راهيم جع...


In [ ]:
new_df['Summary1'][0]

'يقم اثن عشر وظف سسي في عمل هضم . كذلك قبل اثن عشر عصر صفراء التي فرز رار .'

In [ ]:
new_df_list = preprocessSentence(data)

In [ ]:
new_df_list.head()

,Orignal,Summary1,Summary2,Summary3,Summary4,Summary5
0,[اثن عشر انجليزية Duodenum او عفج هو احد جزء ق...,"[يقم اثن عشر وظف سسي في عمل هضم, كذلك قبل اثن ...",[اثن عشر انجليزية Duodenum او عفج هو احد جزء ق...,[يقم اثن عشر وظف سسي في عمل هضم],[واد نشي التي لم يتم فكك في لفم و عدة يقم ثني ...,[اثن عشر انجليزية Duodenum او عفج هو احد جزء ق...
1,[في وحد خدم شرك كهرباء عسر صبح اليوم تقل رقم ا...,[في وحد خدم شرك كهرباء عسر صبح اليوم تقل رقم ا...,[في وحد خدم شرك كهرباء عسر صبح اليوم تقل رقم ا...,[في وحد خدم شرك كهرباء عسر صبح اليوم تقل رقم ا...,[في وحد خدم شرك كهرباء عسر صبح اليوم تقل رقم ا...,[في وحد خدم شرك كهرباء عسر صبح اليوم تقل رقم ا...
2,[سيح هي احد دين سمو التي عبر يسع سيح شخص سسي ف...,[سيح هي احد دين سمو التي عبر يسع سيح شخص سسي ف...,[سيح هي احد دين سمو التي عبر يسع سيح شخص سسي ف...,[سيح هي احد دين سمو التي عبر يسع سيح شخص سسي ف...,"[عبر سيح كثر دين تبع في علم عدد تبع بلغ 2, 1 ب...",[سيح هي احد دين سمو التي عبر يسع سيح شخص سسي ف...
3,[عبر قدس دين قدس نسب دين وحد ثلث سلم سيح يهد ن...,[عبر قدس دين قدس نسب دين وحد ثلث سلم سيح يهد ن...,[اول اسم ثبت لمد قدس هو رسلم قبل خمس الف عام و...,[عبر قدس دين قدس نسب دين وحد ثلث سلم سيح يهد ن...,[ذكر في ارخ يهد سيح عبر ركز لكي دين ممل يهذ ار...,[عبر قدس دين قدس نسب دين وحد ثلث سلم سيح يهد ن...
4,[اعل صدر رسم ان حكم انتقالية رءس دكتور راهيم ج...,[اعل صدر رسم ان حكم انتقالية رءس دكتور راهيم ج...,[اعل صدر رسم ان حكم انتقالية رءس دكتور راهيم ج...,[اعل صدر رسم ان حكم انتقالية رءس دكتور راهيم ج...,[علي صعد اخر اعل قيد في حرك وفق وطن عرق ل وطن ...,[اعل صدر رسم ان حكم انتقالية رءس دكتور راهيم ج...


In [ ]:
new_df_list['Orignal'][133]

['عهد ءسس ثمر امر وبريطانية تدر اصل جوز قيم 3 ريليو دولار باستثمار لير دولار في شرك طقة نظف في حال لحد من خطر التي سبب تغر نخي',
 'وقل رءس كتب خدم درة الة في يفر ستف ستل ان مول ثمر في اي شرع سوء ولد طقة كهرباءية او خدم ورب كثر كفء في محط كهرباء او شرك صنع سير مثل توو التي نتج سير عمل وقد كهرباء',
 'وقل ستل همت شجع شرك علي فكر في بيء',
 'جاء ذلك خلل جمع قمة ءسس ثمر تصد خطر نخي الذي عقد في مقر امم تحد حضر سءل الي من ولي امر مول ثمر كبر بحث كيف عمل مع خطر الة تغر نخي']

In [ ]:
len(new_df_list['Orignal'][133])

4

# Feature extraction and formulation


## Keyphrase extraction

In [ ]:
!pip install git+https://github.com/LIAAD/yake 

  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-ojrfo1kw
  Running command git clone -q https://github.com/LIAAD/yake /tmp/pip-req-build-ojrfo1kw
     |████████████████████████████████| 143kB 16.1MB/s 
     |████████████████████████████████| 655kB 38.9MB/s 
  Created wheel for yake: filename=yake-0.4.2-py2.py3-none-any.whl size=64301 sha256=3692970881c60752988f6fdc49223b53d46e1100266545db837428e4d257b769
  Stored in directory: /tmp/pip-ephem-wheel-cache-2fdvlyc5/wheels/be/35/27/e4ebd54b78c1806ed8b0271ce247fcd91e2bedde35889fbc9b
  Created wheel for segtok: filename=segtok-1.5.7-cp36-none-any.whl size=23258 sha256=be73b18c4fd1a8c5e746dfdf671040159098bc2ec7468cd44eac657328bcbdfb
  Stored in directory: /root/.cache/pip/wheels/15/ee/a8/6112173f1386d33eebedb3f73429cfa41a4c3084556bcee254
  Created wheel for jellyfish: filename=jellyfish-0.7.2-cp36-cp36m-linux_x86_64.whl size=72983 sha256=5a09744d9db1cd388e6d0629f631e73185213637d95635615fbf6e2ad44335cc
  Stored in directory: /roo

In [ ]:
import yake

text = data.iloc[133,0]

# assuming default parameters
simple_kwextractor = yake.KeywordExtractor()
keywords = simple_kwextractor.extract_keywords(text)

print("assuming default parameters")
for kw in keywords:
	print(kw)

# specifying parameters
max_ngram_size = 3
custom_kwextractor = yake.KeywordExtractor(lan="ar", n = max_ngram_size, dedupLim=0.9, dedupFunc='seqm', windowsSize=1, top=20, features=None)

keywords = custom_kwextractor.extract_keywords(text)

print("\nspecifying parameters")
for kw in keywords:
	print(kw)

assuming default parameters
('تريليونات دولار باستثمار', 0.0022186092091954343)
('تعهدت مؤسسات مستثمرة', 0.0025302016573197153)
('أصولا تتجاوز قيمتها', 0.0025302016573197153)
('دولار باستثمار مليار', 0.0027412138035690712)
('باستثمار مليار دولار', 0.0027412138035690712)
('مؤسسات مستثمرة أمريكية', 0.0031273640945779515)
('مستثمرة أمريكية وبريطانية', 0.0031273640945779515)
('أمريكية وبريطانية تدير', 0.0031273640945779515)
('وبريطانية تدير أصولا', 0.0031273640945779515)
('تدير أصولا تتجاوز', 0.0031273640945779515)
('التي تسببها التغيرات', 0.0051441986981571736)
('تسببها التغيرات المناخية', 0.008242613143495878)
('شركات الطاقة النظيفة', 0.00851744943114353)
('المخاطر التي تسببها', 0.013561219145878258)
('تريليونات دولار', 0.014983498131440719)
('تعهدت مؤسسات', 0.017052913676598704)
('تتجاوز قيمتها', 0.017052913676598704)
('دولار باستثمار', 0.018455421033530744)
('مليار دولار', 0.018455421033530744)
('مؤسسات مستثمرة', 0.021014282152231275)

specifying parameters
('تريليونات دولار باستثمار',

In [ ]:
data['Orignal'][0].split('\n')

['\ufeffالإثنا عشري(بالإنجليزية: Duodenum) أو العفج هو أحد أجزاء القناة الهضمية وأول جزء من الأمعاء الدقيقة يصل بين المعدة و الصائم. طوله 25-30 سم وله أربعة قطع تشبه المضلع تنفتح في القطعة الثانية منه كلا من القناتين الصفراوية والمعثكلية ويفصله عن المعدة البواب.يقوم الاثنا عشر بوظيفة أساسية في عملية الهضم.المواد النشوية التي لم يتم تفكيكها في الفم و المعدة يقوم الثنى عشر بإكمال عملية تفكيكها بمساعدة أنزيمات يفرزها البنكرياس كذلك مقاطع البروتينات المسماه ببتونات التي تفككت من البروتينات في المعدة تستمر عملية هضمها في الاثنى عشر بمساعدة أنزيمات أساسية. كذلك يتم تحليل وهضم الدهون وكذا تتحلل وتهضم في الاثنى عشر بواسطة أنزيمات تصل من البنكرياس الفولماريات مثل المواد المضافة للغذاء كاللون والمنكهات وما شابه التي لم تحلل في المعدة والفم.المواد التي تصل من البنكرياس والكبد و المرارة تساعد على تحليل بعض مركبات الغذاء كيميائيا والمواد التي يتم تحليلها كيميائيا هي:بروتينات كروهيدرات ودهون بالاضافة لذلك يتم امتصاص مياة وموادمعدنية الى الدم علما بان المواد المعدنية لا يحل بها تغيير في العملية الهضم

In [ ]:
l = data['Orignal'][0].split('\n')
for s in l : print (len(s))

1169


In [ ]:
l = data['Orignal'][0].split('\n')
for s in l : print (len(s.split('.')))

9


In [ ]:
def key_phrase(document):
  '''
  input : document is a string 
  output : tuples of keywords 
  
  '''
  
  import yake

  # specifying parameters
  max_ngram_size = 3
  custom_kwextractor = yake.KeywordExtractor(lan="ar", n = max_ngram_size, dedupLim=0.9, dedupFunc='seqm', windowsSize=1, top=20, features=None)
  keywords = custom_kwextractor.extract_keywords(text)
  return keywords


## Sentence location feature 

In [ ]:
# to be implmented after figuring paragraph structure 

## Similarity with title feature

In [ ]:
# There is no title available 
title = ' الامارات دولة عظيمة'

In [ ]:
def bag_of_words(all_sentences) :
  '''
  input : preprocessed list of all sentences in a document
  output : return bag of words model array (where sentence i is model[i])
  '''
  from sklearn.feature_extraction.text import CountVectorizer 
  vectorizer = CountVectorizer()
  X = vectorizer.fit_transform(all_sentences)
  return X.toarray()

In [ ]:
def get_bag_of_words_vector(sentence,document_list) :
  '''
  In this model, each sentence Si is represented as an N-dimensional vector Si = {wi1 ..wim}  where wik is the weight of term tk
  that exists in the sentence Si, and n is the number of all possible unique words in the target document (target document is the original document )
  
  '''
  bag_of_words_array = bag_of_words(document_list)

In [ ]:
def cosine_similarity(sentence,title):
  # the sentence and the title are represented using the bag-of-words model
  pass

## Sentence centrality feature


## Sentence length feature


In [ ]:
def outliers_iqr(sentences_length_list):
  '''
  compute interquartile range for all sentences 
  '''
  quartile_1, quartile_3 = np.percentile(sentences_length_list, [25, 75])
  iqr = quartile_3 - quartile_1
  lower_bound = quartile_1 - (iqr * 1.5)
  upper_bound = quartile_3 + (iqr * 1.5)
  return np.where((sentences_length > upper_bound) | (sentences_length < lower_bound))

In [ ]:
def outliers_iqr(sentence_length ,sentences_length_list):
  '''
  compute interquartile range for current sentence
  '''
  quartile_1, quartile_3 = np.percentile(sentences_length_list, [25, 75])
  iqr = quartile_3 - quartile_1
  lower_bound = quartile_1 - (iqr * 1.5)
  upper_bound = quartile_3 + (iqr * 1.5)
  return  1 if (sentence_length > upper_bound) | (sentence_length < lower_bound) else 0

In [ ]:
def sentence_length_score(sentence ,document_list):
  #pdb.set_trace()
  sentences_length = [len(sentence) for sentence in document_list]
  document_max_length = max(sentences_length)
  is_outlier = outliers_iqr(len(sentence) ,sentences_length)
  
  if is_outlier : 
    return 0
  else :
    return len(sentence) / document_max_length
    

## Cue-phrases feature


In [ ]:
cue_phrases = ["الاهم" , "بالتحديد" , "كنتيجة" , "الافضل "]

In [ ]:
def cue_phrases_score(sentence ,document_list , cue_phrases):
  
  number_of_sentence_cue_phrases = 0
  for cue in cue_phrases : 
    for word in sentence :
      if cue == word : 
        number_of_sentence_cue_phrases+=1
        
  number_of_document_list_cue_phrases = 0
  for s in document_list :
    for cue in cue_phrases : 
      for word in s :
        if cue == word : 
          number_of_document_list_cue_phrases+=1
          
  return number_of_sentence_cue_phrases / number_of_document_list_cue_phrases


## Strong Word Feature

In [ ]:
strong_words = ["وثّق","أكّد "]

In [ ]:
def strong_words_score(sentence ,document_list , strong_words):
  
  number_of_strong_words = 0
  for cue in strong_words : 
    for word in sentence :
      if cue == word : 
        number_of_strong_words+=1
        
  number_of_document_list_strong_words = 0
  for s in document_list :
    for cue in cue_phrases : 
      for word in s :
        if cue == word : 
          number_of_document_list_strong_words+=1
          
  return number_of_strong_words / number_of_document_list_strong_words

## Existence of numerical data

In [ ]:
# should handle both arabic and english numbers -> numbers in data already english maybe because of utf8 encoding 

In [ ]:
s = [['٣٣٣', '٢', 'يا', 'رباااااااه' ,'٣']]

In [ ]:
def convert_arabic_to_english(document_list) :
  eastern_to_western = {"٠":"0","١":"1","٢":"2","٣":"3","٤":"4","٥":"5","٦":"6","٧":"7","٨":"8","٩":"9"}
  
  for i in  range(len(document_list)) : 
    for j in range(len(document_list[i])):
      if document_list[i][j][0] in eastern_to_western :
        document_list[i][j]  = ''.join([eastern_to_western[c] for c in document_list[i][j]])
  return document_list

In [ ]:
convert_arabic_to_english(s)

[['333', '2', 'يا', 'رباااااااه', '3']]

In [ ]:
def numerical_data_score(sentence ,document_list):
  
  numbers_count_sentence = 0
  for word in sentence :
    if word : 
      number_of_strong_words+=1
        
  number_of_document_list_strong_words = 0
  for s in document_list :
    for cue in cue_phrases : 
      for word in s :
        if cue == word : 
          number_of_document_list_strong_words+=1
          
  return number_of_strong_words / number_of_document_list_strong_words

# Score-Based Algorithm

In [ ]:
features = [sentence_length_score ,cue_phrases_score,strong_words_score]

In [ ]:
dubai = data.iloc[22,0]
dubai = noramlize(dubai)
#dubai = stopWordRmove(dubai)
#dubai = stemmingSentence(dubai)
dubai = stopWordRmoveSentence(dubai)


In [ ]:
len(dubai)

8

In [ ]:
def score(document_list) :
  lst = []
  ordered_list = []
  max_legnth_summary = 4
  S = document_list.copy()
  summary  = []
  sentence_scores = []
  for index,sentence in enumerate(document_list) :
    total_score = 0
    score = sentence_length_score(sentence ,document_list)
    total_score += score
    sentence_scores.append((index,total_score))
    
  
  ordered_list = sorted(sentence_scores,key =  lambda x : x[1] ,reverse = True)
  
  i = 0
  while len(summary) < max_legnth_summary : 
    summary.append(ordered_list[i])
    i+=1
  
  last_summary = sorted(summary,key =  lambda x : x[0])
  return last_summary
    
    
  
  
    
    
  

In [ ]:
score_list = score(dubai)
score_list

[(1, 0.8222222222222222),
 (4, 0.8222222222222222),
 (5, 0.9111111111111111),
 (6, 1.0)]

In [ ]:
sum_list = [dubai[x] for (x,y) in score_list]
text_list = [' '.join(s) for s in sum_list]
text_list

['ان اصل نشاة الكاراتيه في اليابان علي جزيرة اوكيناوا وسبب موقعها الجغرافي الممتاز حيث قربها علي الصين ادي الي تفاعل تجارة ناجحة مع جيرانها الصينيين مما جعل بعضا من ثقافتة متاثر قليلا في ثقافة الصين وفنونها القتالية',
 'وعندما ارادت اليابان التوحد في فترة ميجي واسقاط الحكم الاقطاعي في كانت هناك بعض المناطق في اليابان لا بد وان توحد بالقوة ومن هذه المناطق ارخبيل ريوكو الذي يوجد فيه ولاية اوكيناوا التي ظهر فيها فن الكاراتيه',
 'فقد هاجمتها قوات هاءلة من الساموراي عام 1600م واسقطت النظام الاقطاعي الحاكم للارخبيل وعند ذلك اجرت قوات الساموراي عملية حظر السلاح علي سكان الجزيرة خوفا من الثورات ضدهم فاراد الاوكيناويين ان يستعملو اعضاء جسدهم كسلاح للدفاع عن النفس علي جزيرة اوكيناوا',
 'وبعد ذلك وحدتها الحكومة اليابانية في فترة حكم الامبراطور ميجي عام 1879م وبعد ان ارجع ارخبيل ريوكو وجزيرة اوكيناوا الي الحكم الياباني اراد فوناكشي ومجموعة من خبراء فنون القتال اليابانيه في طوكيو ان يجعلوا الكاراتيه فن قتالي عسكري ياباني مءكد من جمعية اليابان لفنون القتال']

#Evaluation

In [ ]:
# all=[]
# for i in range(len(data.columns[1:])):
#   all.append(data[data.columns[i][22]])

# all
# all=[]
# for i in range(1,6):
#   sumar_i = data['summary{}'.format(i)][22]
#   all.append(sumar_i)


In [ ]:
print(data.columns[1:])

Index(['Summary1', 'Summary2', 'Summary3', 'Summary4', 'Summary5'], dtype='object')


#golden_summary

In [ ]:
sumar_1= data['Summary1'][21]
sumar_2= data['Summary2'][21]
sumar_3= data['Summary3'][21]
sumar_4= data['Summary4'][21]
sumar_5= data['Summary5'][21]
all_summaries = [sumar_1,sumar_2,sumar_3,sumar_4,sumar_5]


In [ ]:
res=[]
for sam in range(len(all_summaries)):
  res.append(all_summaries[sam].split('.'))
  #res[sam].remove('')
  

res

[[' وتم منح الجوائز في عدد من الفئات إلى المطورين الذين تمكنوا من استثمار تقنيات مايكروسوفت على أكمل وجه بما فيه فائدة قطاع التجزئة والعملاء',
  ' وسبق في عام 2002 أن فازت "دو كونت" بجائزة "مايكروسوفت فيوجن" التي أقيمت في لوس أنجلوس عن حل الشرطة الإلكترونية ضمن فئة "أفضل الحلول التنسيقية"',
  ' وقد جاء هذا الفوز نتيجة لالتزامنا بتطوير أفضل الحلول التقنية لهذه المنطقة، وللدعم الذي استمديناه من الرؤية الاستراتيجية لشرطة دبي والتطور الكبير الذي تتميز به تقنيات مايكروسوفت المبتكرة"',
  ''],
 [' ولكن لجنة التحكيم المستقلة التي تضم تجاراً وصحفيين ومحللين من قطاع التجزئة قررت ترقية حل "دو كونت" المتطور إلى فئة "أفضل ابتكار" التي استحقت الشركة عنها الجائزة الأولى بجدارة',
  ''],
 ['وتم منح الجوائز في عدد من الفئات إلى المطورين الذين تمكنوا من استثمار تقنيات مايكروسوفت على أكمل وجه بما فيه فائدة قطاع التجزئة والعملاء\nوقد تمكنت دوكونت، الشركة الرائدة في مجال تطبيقات الإنترنت السلكية واللاسلكية في منطقة الشرق الأوسط، من الفوز بجائزة هذه الفئة عن حل الحوسبة النقالة الذي طورته لصالح شرطة دبي العام

In [ ]:
from collections import defaultdict
dic = defaultdict(int)

golden_list = []
for i in range (len (res)):
  for l in range(len(res[i])):
    dic[res[i][l]]+=1

dic

defaultdict(int,
            {'': 2,
             '\n\n\n \n\n': 1,
             '\n\n \n\n': 2,
             '  وعليه، فإننا نهنئ "دوكونت" ونتمنى لهم المزيد من النجاح"': 1,
             ' من جانبه، أضاف عبد اللطيف الملا، مدير عام مايكروسوفت جنوب الخليج: "حظيت "دوكونت" من خلال الحل المبتكر الذي وفرته لشرطة دبي باستخدام تقنيات مايكروسوفت المتطورة بالفرصة للتنافس مع مطوري البرمجيات والأنظمة الحاسوبية العالميين واكتساب شهرة عالمية': 3,
             ' وتم منح الجوائز في عدد من الفئات إلى المطورين الذين تمكنوا من استثمار تقنيات مايكروسوفت على أكمل وجه بما فيه فائدة قطاع التجزئة والعملاء': 1,
             ' وسبق في عام 2002 أن فازت "دو كونت" بجائزة "مايكروسوفت فيوجن" التي أقيمت في لوس أنجلوس عن حل الشرطة الإلكترونية ضمن فئة "أفضل الحلول التنسيقية"': 1,
             ' وقال الدكتور عمر بن سليمان، الرئيس التنفيذي لمدينة دبي للإنترنت: "تقدم هذه الجائزة خير دليل على أن البرمجيات التجارية توفر للمطورين المنصة المثالية لإيجاد حلول مبتكرة للتغلب على التحديات الفعلية التي تواجه قطاع الأعمال"': 3,
   

In [ ]:
for k,v in dic.items():
  if v>=2 : golden_list.append(k)

golden_list    

['',
 ' وقال الدكتور عمر بن سليمان، الرئيس التنفيذي لمدينة دبي للإنترنت: "تقدم هذه الجائزة خير دليل على أن البرمجيات التجارية توفر للمطورين المنصة المثالية لإيجاد حلول مبتكرة للتغلب على التحديات الفعلية التي تواجه قطاع الأعمال"',
 ' من جانبه، أضاف عبد اللطيف الملا، مدير عام مايكروسوفت جنوب الخليج: "حظيت "دوكونت" من خلال الحل المبتكر الذي وفرته لشرطة دبي باستخدام تقنيات مايكروسوفت المتطورة بالفرصة للتنافس مع مطوري البرمجيات والأنظمة الحاسوبية العالميين واكتساب شهرة عالمية',
 'دبي: أعلنت مايكروسوفت جنوب الخليج عن فوز شركة" دوكونت"، أحد أهم شركاء مايكروسوفت في الشرق الأوسط بجائزة "أفضل ابتكار"، وذلك في إطار مشاركتها في جوائز "مطوري تطبيقات قطاع التجزئة" لعام 2003 التي أقيمت في باريس\nوتم منح الجوائز في عدد من الفئات إلى المطورين الذين تمكنوا من استثمار تقنيات مايكروسوفت على أكمل وجه بما فيه فائدة قطاع التجزئة والعملاء\nوقد تمكنت دوكونت، الشركة الرائدة في مجال تطبيقات الإنترنت السلكية واللاسلكية في منطقة الشرق الأوسط، من الفوز بجائزة هذه الفئة عن حل الحوسبة النقالة الذي طورته لصالح شرطة دب

In [ ]:
!pip install Rouge

In [ ]:
orgnl_doc = data['Orignal'][22].replace("\ufeff","")
orgnl_doc

'الكارتيه نوع من أنواع الفنون القتالية اليابانية و تستخدم فيه الأيدي والأقدام والركب و المرافق كأسلحة.\nأن أصل نشأة الكاراتيه في اليابان على جزيرة اوكيناوا وسبب موقعها الجغرافى الممتاز حيث قربها على الصين ادى إلى تفاعل تجارة ناجحة مع جيرانها الصينيين مما جعل بعضا من ثقافتة متأثر قليلا في ثقافة الصين وفنونها القتالية.\nكان الاوكيناويين يسافرون إلى الصين لتعلم فنون القتال وتحديدا الوشوو حيث طورو الكاراتيه من فنونهم القتالية المحلية وبعض من فنون قتالية صينة وهي الوشوو بتحديد.\nاوكيناوا كانت محكومة في تلك الفترة بنظام اقطاعى حيث قد كانت فترة الاقطاعيين في اليابان وحيث كانت الحروب الاهلية كثيرة في اليابان.\nوعندما ارادت اليابان التوحد في فترة ميجى واسقاط الحكم الاقطاعى في كانت هناك بعض المناطق في اليابان لا بد وان توحد بالقوة ومن هذه المناطق ارخبيل ريوكو الذي يوجد فيه ولاية اوكيناوا التي ظهر فيها فن الكاراتيه.\nفقد هاجمتها قوات هائلة من الساموراى عام 1600م واسقطت النظام الاقطاعى الحاكم للارخبيل وعند ذلك اجرت قوات الساموراى عملية حظر السلاح على سكان الجزيرة خوفا من الثورات ضدهم فاراد الاوكين

In [ ]:
golden_list_str = "".join(str(x) for x in golden_list)
golden_list_str

' وقال الدكتور عمر بن سليمان، الرئيس التنفيذي لمدينة دبي للإنترنت: "تقدم هذه الجائزة خير دليل على أن البرمجيات التجارية توفر للمطورين المنصة المثالية لإيجاد حلول مبتكرة للتغلب على التحديات الفعلية التي تواجه قطاع الأعمال" من جانبه، أضاف عبد اللطيف الملا، مدير عام مايكروسوفت جنوب الخليج: "حظيت "دوكونت" من خلال الحل المبتكر الذي وفرته لشرطة دبي باستخدام تقنيات مايكروسوفت المتطورة بالفرصة للتنافس مع مطوري البرمجيات والأنظمة الحاسوبية العالميين واكتساب شهرة عالميةدبي: أعلنت مايكروسوفت جنوب الخليج عن فوز شركة" دوكونت"، أحد أهم شركاء مايكروسوفت في الشرق الأوسط بجائزة "أفضل ابتكار"، وذلك في إطار مشاركتها في جوائز "مطوري تطبيقات قطاع التجزئة" لعام 2003 التي أقيمت في باريس\nوتم منح الجوائز في عدد من الفئات إلى المطورين الذين تمكنوا من استثمار تقنيات مايكروسوفت على أكمل وجه بما فيه فائدة قطاع التجزئة والعملاء\nوقد تمكنت دوكونت، الشركة الرائدة في مجال تطبيقات الإنترنت السلكية واللاسلكية في منطقة الشرق الأوسط، من الفوز بجائزة هذه الفئة عن حل الحوسبة النقالة الذي طورته لصالح شرطة دبي العام الماضي\n

In [ ]:
from rouge import Rouge 


rouge = Rouge()
scores = rouge.get_scores(orgnl_doc, golden_list_str)
print(scores)

[{'rouge-1': {'f': 0.07868851970889576, 'p': 0.06818181818181818, 'r': 0.09302325581395349}, 'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-l': {'f': 0.06266660437269503, 'p': 0.056818181818181816, 'r': 0.07751937984496124}}]


In [ ]:
!git clone https://github.com/motazsaad/khoja-stemmer-command-line.git

Cloning into 'khoja-stemmer-command-line'...
remote: Enumerating objects: 76, done.
remote: Total 76 (delta 0), reused 0 (delta 0), pack-reused 76
Unpacking objects: 100% (76/76), done.


In [ ]:
ls

COPYRIGHT         khoja-stemmer-command-line/  README
EASC-ISO-Arabic/  list.txt                     SOURCES.CSV
EASC-UTF-8/       myfile.txt                   test_in.txt


In [ ]:
cd khoja-stemmer-command-line

/content/EASC/khoja-stemmer-command-line


In [ ]:
!ls

In [ ]:
!pwd

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
ls

In [ ]:
!cat test_in.txt  

In [ ]:
golden_list_str

' وقال الدكتور عمر بن سليمان، الرئيس التنفيذي لمدينة دبي للإنترنت: "تقدم هذه الجائزة خير دليل على أن البرمجيات التجارية توفر للمطورين المنصة المثالية لإيجاد حلول مبتكرة للتغلب على التحديات الفعلية التي تواجه قطاع الأعمال" من جانبه، أضاف عبد اللطيف الملا، مدير عام مايكروسوفت جنوب الخليج: "حظيت "دوكونت" من خلال الحل المبتكر الذي وفرته لشرطة دبي باستخدام تقنيات مايكروسوفت المتطورة بالفرصة للتنافس مع مطوري البرمجيات والأنظمة الحاسوبية العالميين واكتساب شهرة عالميةدبي: أعلنت مايكروسوفت جنوب الخليج عن فوز شركة" دوكونت"، أحد أهم شركاء مايكروسوفت في الشرق الأوسط بجائزة "أفضل ابتكار"، وذلك في إطار مشاركتها في جوائز "مطوري تطبيقات قطاع التجزئة" لعام 2003 التي أقيمت في باريس\nوتم منح الجوائز في عدد من الفئات إلى المطورين الذين تمكنوا من استثمار تقنيات مايكروسوفت على أكمل وجه بما فيه فائدة قطاع التجزئة والعملاء\nوقد تمكنت دوكونت، الشركة الرائدة في مجال تطبيقات الإنترنت السلكية واللاسلكية في منطقة الشرق الأوسط، من الفوز بجائزة هذه الفئة عن حل الحوسبة النقالة الذي طورته لصالح شرطة دبي العام الماضي\n

In [ ]:
!echo 'امارة دبي هي ثاني الامارات المكونة لدولة الامارات العربية المتحدة وعاصمتها مدينة دبي . تشكل هذه الامارة مركزا هاما للمال والاعمال في العالم ووجهة سياحية يقصدها الملايين من السياح سنويا.دبي هي العاصمة الاقتصادية للامارات العربية المتحدة وقد تطورت تطورا كبيرا خلال السنوات الماضية . الاقتصاد الحر والنشط في الامارة وعدم وجود نظام ضريبي لعب دورا كبيرا في جذب المستثمرين من جميع انحاء العالم . وتقع امارة دبي بين امارتي ابو ظبي والشارقة . واهل امارة دبي ينحدرون من قباءل عربية متنوعة علي راسها قبيلة ال بو فلاسه التي تنحدر منها اسرة ال مكتوم الحاكمة . وتقطنها قباءل بني كعب و ال بو فلاح وال بو مهير والسودان والشوامس والبلوش والمناصير والرميثات والشحوح وغيرهم . وبها عواءل كثيرة من اصول افريقية وفارسية . ودين اهالي دبي هو الاسلام علي نهج اهل السنة والجماعة والمذهب الرسمي في دبي هو المذهب المالكي.ال مكتوم هم حكام دبي . وهم من ال بو فلاسه من بني ياس . حاكمها الان هو الشيخ محمد بن راشد ال مكتوم . وهو ايضا ناءب لرءيس الدولة و رءيس لمجلس الوزراء في الحكومة الاتحادية . وناءبيه في الحكم هما شقيقه الشيخ حمدان بن راشد ال مكتوم وزير المالية والصناعة والشيخ مكتوم بن محمد بن راشد ال مكتوم . بينما يتولي منصب ولاية العهد بالامارة الشيخ حمدان بن محمد بن راشد ال مكتوم رءيس المجلس التنفيذي للامارة . يراس المجلس التنفيذي لحكومة دبي الشيخ حمدان بن محمد بن راشد ال مكتوم . ويجمع هذا المجلس في عضويته جميع مدراء الدواءر في حكومة دبي حيث يعقدوت اجتماعاتهم الدورية لتسيير شءون الامارة .' > myfile.txt


In [ ]:
!cat myfile.txt

امارة دبي هي ثاني الامارات المكونة لدولة الامارات العربية المتحدة وعاصمتها مدينة دبي . تشكل هذه الامارة مركزا هاما للمال والاعمال في العالم ووجهة سياحية يقصدها الملايين من السياح سنويا.دبي هي العاصمة الاقتصادية للامارات العربية المتحدة وقد تطورت تطورا كبيرا خلال السنوات الماضية . الاقتصاد الحر والنشط في الامارة وعدم وجود نظام ضريبي لعب دورا كبيرا في جذب المستثمرين من جميع انحاء العالم . وتقع امارة دبي بين امارتي ابو ظبي والشارقة . واهل امارة دبي ينحدرون من قباءل عربية متنوعة علي راسها قبيلة ال بو فلاسه التي تنحدر منها اسرة ال مكتوم الحاكمة . وتقطنها قباءل بني كعب و ال بو فلاح وال بو مهير والسودان والشوامس والبلوش والمناصير والرميثات والشحوح وغيرهم . وبها عواءل كثيرة من اصول افريقية وفارسية . ودين اهالي دبي هو الاسلام علي نهج اهل السنة والجماعة والمذهب الرسمي في دبي هو المذهب المالكي.ال مكتوم هم حكام دبي . وهم من ال بو فلاسه من بني ياس . حاكمها الان هو الشيخ محمد بن راشد ال مكتوم . وهو ايضا ناءب لرءيس الدولة و رءيس لمجلس الوزراء في الحكومة الاتحادية . وناءبيه في الحكم هما شقيقه الشيخ حم

In [ ]:
import subprocess
subprocess.call(['java', '-jar', '.khoja-stemmer-command-line.jar'])

1

In [ ]:
!java -jar khoja-stemmer-command-line.jar myfile.txt test-out.txt

In [ ]:
cd /content/EASC/khoja-stemmer-command-line

/content/EASC/khoja-stemmer-command-line


In [ ]:
!cat test-out.txt

مور دوب ثاني مرأ كون دول مرأ عرب حدد عصم مدين دوب . شكل مور مركز ها مول عمل علم وجه سوح قصد لون سوح سنويا.دبي عصم قصد مرأ عرب حدد طور طور كبر نوت مضي . قصد حري نشط مور عدم نظم ضرب لعب دور كبر جذب ثمر حاء علم . قعي مور دوب مور بوأ ظبي شرق . وهل مور دوب حدر قباءل عرب نوع علي روس قبل ل بوأ فلس تنحدر أسر ل كوم حكم . قطن قباءل بني كعب ل بوأ فلح ويل بوأ هور سود شمس بلوش نصر رمث شحح غير . بها عواءل كثر صول فرق فرس . دون هول دوب سلم علي نهج أهل وسن جمع مذهب رسم دوب مذهب مالكي.ال كوم حكم دوب . وهم ل بوأ فلس بني ياس . حكم شيخ حمد رشد ل كوم . ناءب لرءيس دول رءيس جلس وزر حكم حدأ . وناءبيه حكم هما شقق شيخ حمد رشد ل كوم وزر ملأ صنع شيخ كوم حمد رشد ل كوم . ولي نصب لوي عهد مور شيخ حمد حمد رشد ل كوم رءيس جلس نفذ مور . روس جلس نفذ حكم دوب شيخ حمد حمد رشد ل كوم . جمع جلس عضويته درأ دواءر حكم دوب يعقدوت جمع دور سور شءون مور . 
